# Pandas IO İşlemleri
![alt text](Pandas_read_to.png)

In [248]:
import numpy as np
import pandas as pd

In [249]:
#Yas kolonundan önce boşluk var.
#Bir satır atlanmış.
#Ayraçlar |
#None değerler var. Ali de kilo eksik.
#? işareti None yerine kullanışmış.
veri = """
Ad|Soyad| Yas|Tarih|Kilo
Abdullah|Kise|66|2025-05-06|70,5

Ali|Uçan|22|2024-12-06|
Veli|Kaçan|35|?|?
"""

In [250]:
import io

vh = io.StringIO(veri)
vh.read()

'\nAd|Soyad| Yas|Tarih|Kilo\nAbdullah|Kise|66|2025-05-06|70,5\n\nAli|Uçan|22|2024-12-06|\nVeli|Kaçan|35|?|?\n'

In [251]:
#248.0+ bytes
# 215.0+ bytes
df = pd.read_csv(io.StringIO(veri),sep="|"
                 #,usecols=["Ad","Soyad"]
                 ,na_values="?"
                 ,parse_dates=["Tarih"] #bu kolon tarih tipine çevrilir
                 ,decimal="," #kilo alanında ondalık kısım için , kullanıldı.
                 ,dtype={
                         "Ad":str,
                         "Soyad":str,
                         " Yas":np.uint8,
                         "Kilo":np.float32
                        }
                ,converters={
                             #"Soyad":str.upper #bizden bir fonksiyon bekler. kolona uygular.
                             #"Soyad":lambda v:f"{v :.3s}***"
                             "Soyad":"{:.3s}***".format
                            }
                )
#
#df.info()
df

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_15104\3155534242.py:3: ParserWarning: Both a converter and dtype were specified for column Soyad - only the converter will be used.
  df = pd.read_csv(io.StringIO(veri),sep="|"


,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kis***,66,2025-05-06,70.5
1,Ali,Uça***,22,2024-12-06,NaN
2,Veli,Kaç***,35,NaT,NaN


### CSV Dosyası Olarak Kaydedelim

In [252]:
import os

os.makedirs("databases",exist_ok=True)

In [253]:
df.to_csv("./databases/kisiler.csv",index=False)

#
os.listdir("./databases")

['kisiler.csv', 'kisiler.xlsx']

In [254]:
!cd

c:\Users\Abdullah\Documents\GitHub\TSPython_20250929\Gun5


## Excel Olarak Kaydedelim

In [255]:
df.to_excel("./databases/kisiler.xlsx",sheet_name="Personeller",index=False)

## Excelden Okuyalım

In [256]:
df_dict = pd.read_excel("./databases/kisiler.xlsx"
              #,sheet_name="Personeller"
              ,sheet_name=None #bütün sayfaları getirir.
              ,
              )

df_dict

#
df_dict["Personeller"] #df olarak elde ettik.

,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kis***,66,2025-05-06,70.5
1,Ali,Uça***,22,2024-12-06,NaN
2,Veli,Kaç***,35,NaT,NaN


In [257]:
#import re
#dilersek regex ile arama yapabiliriz. belli bir kalıptaki keye ulaşabiliri.

#
key = [k for k in df_dict.keys() if k.__contains__("Per")][0]

df_dict.get(key)

,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kis***,66,2025-05-06,70.5
1,Ali,Uça***,22,2024-12-06,NaN
2,Veli,Kaç***,35,NaT,NaN


## RDMBS (MSSQL, Oracle) Bağlanma
![alt text](RDBMSe_Baglanti.png)

#### Bağlantı Cümleleri:
* **DBAPI**:
  * `Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=AdventureWorks2012;Trusted_Connection=yes;`
* **SqlAlchemy (URI)**:
  * `mssql+pyodbc://localhost/AdventureWorks2012?driver=ODBC Driver 17 for SQL Server`


### DBAPI

In [258]:
#Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=AdventureWorks2012;Trusted_Connection=yes;
import pyodbc

con = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=AdventureWorks2012;Trusted_Connection=yes;")

#
df = pd.read_sql_query("SELECT Name,Color,ListPrice FROM Production.Product",con)
df.head()

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_15104\271073095.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT Name,Color,ListPrice FROM Production.Product",con)


,Name,Color,ListPrice
0,Adjustable Race,None,0.0
1,Bearing Ball,None,0.0
2,BB Ball Bearing,None,0.0
3,Headset Ball Bearings,None,0.0
4,Blade,None,0.0


### SqlAlchemy

In [259]:
#mssql+pyodbc://localhost/AdventureWorks2012?driver=ODBC Driver 17 for SQL Server
import sqlalchemy as sqla

eng = sqla.create_engine("mssql+pyodbc://localhost/AdventureWorks2012?driver=ODBC Driver 17 for SQL Server")

#
df = pd.read_sql("SELECT Name,Color,ListPrice FROM Production.Product",eng)
#df = pd.read_sql_table(table_name="Product", schema="Production",con=eng)
df.head()

,Name,Color,ListPrice
0,Adjustable Race,None,0.0
1,Bearing Ball,None,0.0
2,BB Ball Bearing,None,0.0
3,Headset Ball Bearings,None,0.0
4,Blade,None,0.0


## Webden Bir Okuma

In [275]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_html("https://www.bddk.org.tr/BultenHaftalik/"
                 ,decimal=","
                 ,converters={
                                "TP":lambda tp: tp.replace(".","").replace(",","."),
                                "YP":lambda tp: tp.replace(".","").replace(",",".")
                 })
df.__len__() #6 tane tablo okumuş

type(df) #list

list

In [261]:
dff = df[3].copy()
dff.columns[1]

'Sektör / Krediler ( 3 Ekim 2025 Cuma  ) (Milyon TL)'

In [262]:
import re

#'Sektör / Krediler ( 3 Ekim 2025 Cuma  ) (Milyon TL)'
#yukarıdkai metinde geçen tarihi regex ile bulalım.
tarih = re.findall(r"\d{1,2}\s\w+\s\d{4}",dff.columns[1])[0]
tarih

'3 Ekim 2025'

In [263]:
dff = dff.iloc[:,1:4].copy()

In [264]:
#float("13.111.956,82".replace(".","").replace(",","."))

In [274]:
dff.columns = ["Krediler","TP","YP"]
dff = dff.astype({
            "TP":np.float64,
            "YP":np.float64
})

#dff.info()
dff.TP.sum()
dff.sum()

Krediler    Toplam Krediler (2+10)Tüketici Kredileri ve Bi...
TP                                                50690617.05
YP                                                20330221.26
dtype: object

In [ ]:
dff.to_excel("Krediler.xlsx",sheet_name=tarih,index=False)